# Лабороторна робота 3
# Злиття датафреймів, агрегування даних та візуалізація даних
## Мета: Вивчення функцій Pandas для злиття, агрегування та візуалізації даних

1. Загрузити файл з даними про споживання енергії “En_In.xls”, який являє собою перелік показників енергозабезпечення та виробництва відновлюваної електроенергії, і ввести їх в DataFrame.

In [34]:
import pandas as pd

file_path = 'En_In.xlsx'
en_data = pd.read_excel(file_path, skiprows=2, engine='openpyxl')
en_data = en_data.iloc[:, 2:]
en_data = en_data.drop(index=6)
en_data.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
en_data = en_data[~en_data['Country'].str.contains('Energy Supply|Choose a country|Last update', na=True)]
en_data = en_data.dropna(subset=['Country', 'Energy Supply'])

display(en_data)

,Country,Energy Supply,Energy Supply per Capita,% Renewable
8,Afghanistan,321,10,78.66928
15,Afghanistan,321,10,78.66928
16,Albania,102,35,100
17,Algeria,1959,51,0.55101
18,American Samoa,...,...,0.641026
...,...,...,...,...
237,Viet Nam,2554,28,45.32152
238,Wallis and Futuna Islands,0,26,0
239,Yemen,344,13,0
240,Zambia,400,26,99.71467


2. Переведіть дані із стовпчику ‘Energy Supply’ в гДж (Примітка: в 1ПДж = 1000000 ГДж). Для всіх країн, у яких відсутні дані (наприклад, дані з "....."), переконайтеся, що це відображається як значення np.NaN.

In [35]:
en_data['Energy Supply'] = pd.to_numeric(en_data['Energy Supply'], errors='coerce')
en_data['Energy Supply'] = en_data['Energy Supply'] * 1_000_000

3. Перейменуйте наступний список країн: "Republic of Korea": "South Korea", "United States of America": "United States", "United Kingdom of Great Britain and Northern Ireland": "United Kingdom", "China, Hong Kong Special Administrative Region": "Hong Kong"

In [36]:
en_data['Country'] = en_data['Country'].replace({
    "Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"
})

4. Існує також декілька країн, що мають в назві цифри та/або дужки. Обов’язково видаліть їх, напр. 'Bolivia (Plurinational State of)'повинна бути ‘Bolivia’, ‘Switzerland17’ повинна бути Switzerland’

In [37]:
import re

def clean_country_name(name):
    return re.sub(r'\d+|\s*\(.*?\)', '', name).strip()

en_data['Country'] = en_data['Country'].apply(clean_country_name)

In [17]:
display(data.loc[data['Country'].isin(['American Samoa', 'South Korea', 'Bolivia'])])

,Country,Energy Supply,Energy Supply per Capita,% Renewable
18,American Samoa,NaN,...,0.641026
39,Bolivia,3.360000e+08,32,31.47712
179,South Korea,1.100700e+10,221,2.279353


5. Далі, завантажте дані про ВВП з файлу „gpd.csv”, що містить дані Світового банку про ВВП країн від 1960 до 2015 року.
Обов’язково не включайте заголовок до датафрейму і перейменуйте наступний список країн:
"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran", 
"Hong Kong SAR, China": "Hong Kong"

In [29]:
gdp_data = pd.read_csv('gpd.csv')

gdp_data['Country Name'] = gdp_data['Country Name'].replace({
    "Korea, Rep.": "South Korea",
    "Iran, Islamic Rep.": "Iran",
    "Hong Kong SAR, China": "Hong Kong"
})

display(gdp_data.head(1))

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Aruba,ABW,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.467704e+09,NaN,NaN,NaN,NaN,NaN


6. Завантажте дані з файлу „scimagojr.xlsx”, який класифікує країни на основі їхніх публікацій в журналах у галузі енергетичного машинобудування та енергетичних технологій.

In [30]:
scimagojr_data = pd.read_excel('scimagojr.xlsx', engine='openpyxl')

display(scimagojr_data)

,Rank,Country,Documents,Citable documents,Citations,Self-citations,Citations per document,H index
0,1,China,127050,126767,597237,411683,4.70,138
1,2,United States,96661,94747,792274,265436,8.20,230
2,3,Japan,30504,30287,223024,61554,7.31,134
3,4,United Kingdom,20944,20357,206091,37874,9.84,139
4,5,Russian Federation,18534,18301,34266,12422,1.85,57
...,...,...,...,...,...,...,...,...
186,187,Guyana,1,1,0,0,0.00,0
187,188,Christmas Island,1,1,0,0,0.00,0
188,189,Reunion,1,1,2,1,2.00,1
189,190,Saint Lucia,1,1,0,0,0.00,0


7. Приєднайте три набори даних із завдань 1-6 до нового набору даних (використовуючи перетин назв країн).
- Використовуйте лише дані про ВВП за останні 10 років (2006-2015 рр.) і лише 15 найкращих країн за рейтингом Scimagojr (Rank від 1 до 15)
- Індексом цього DataFrame повинна бути назва країни, а стовпцями мають бути ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

In [42]:
gdp_data.rename(columns={gdp_data.columns[0]: 'Country'}, inplace=True)

columns_needed = ['Country'] + [str(year) for year in range(2006, 2016)]
gdp_data = gdp_data[columns_needed]

top_countries = scimagojr_data[scimagojr_data['Rank'] <= 15]

merged_data = pd.merge(top_countries, en_data, on='Country')
final_data = pd.merge(merged_data, gdp_data, on='Country')

final_data.set_index('Country', inplace=True)

final_data = final_data[['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 
                         'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', 
                         '% Renewable'] + [str(year) for year in range(2006, 2016)]]

display(final_data)

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93,19.75491,3.992330e+12,4.559040e+12,4.997780e+12,5.459250e+12,6.039660e+12,6.612490e+12,7.124980e+12,7.672450e+12,8.230120e+12,8.798000e+12
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149,10.23282,5.496540e+12,5.617040e+12,5.558530e+12,5.251310e+12,5.498720e+12,5.473740e+12,5.569100e+12,5.644660e+12,5.642880e+12,5.669560e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214,17.28868,1.385790e+12,1.504070e+12,1.583000e+12,1.459200e+12,1.524920e+12,1.589940e+12,1.645880e+12,1.666930e+12,1.678710e+12,1.616150e+12
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296,61.94543,1.564470e+12,1.596740e+12,1.612710e+12,1.565140e+12,1.613410e+12,1.664090e+12,1.693130e+12,1.730690e+12,1.773490e+12,1.792610e+12
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165,17.90153,3.332890e+12,3.441560e+12,3.478810e+12,3.283340e+12,3.417300e+12,3.542370e+12,3.556720e+12,3.567320e+12,3.624390e+12,3.685560e+12
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26,14.96908,1.265890e+12,1.374870e+12,1.428360e+12,1.549480e+12,1.708460e+12,1.821870e+12,1.924240e+12,2.051980e+12,2.200620e+12,2.367210e+12
France,9,13153,12973,130632,28601,9.93,114,1.059700e+10,166,17.02028,2.607840e+12,2.669420e+12,2.674640e+12,2.595970e+12,2.646990e+12,2.702030e+12,2.706970e+12,2.722570e+12,2.729630e+12,2.761190e+12
South Korea,10,11983,11923,114675,22595,9.57,104,1.100700e+10,221,2.279353,9.410200e+11,9.924320e+11,1.020510e+12,1.027730e+12,1.094500e+12,1.134800e+12,1.160810e+12,1.194430e+12,1.234340e+12,1.266580e+12
Italy,11,10964,10794,111850,26661,10.20,106,6.530000e+09,109,33.66723,2.202170e+12,2.234630e+12,2.211150e+12,2.089940e+12,2.125180e+12,2.137440e+12,2.077180e+12,2.040870e+12,2.033870e+12,2.049320e+12


8. Створіть функцію, щоб визначити середній ВВП країн за останні 10 років.

In [47]:
def average_gdp_sorted(df):
    gdp_columns = [str(year) for year in range(2006, 2016)]
    avg_gdp_series = df[gdp_columns].mean(axis=1)
    avg_gdp_series.index = df.index  # Зберігаємо індекс (назви країн)
    avg_gdp_series.name = 'Average GDP (2006-2015)'
    return avg_gdp_series.sort_values(ascending=False)

average_gdp_result = average_gdp_sorted(final_data)

display(average_gdp_result)

Country
China                 6.348610e+12
Japan                 5.542208e+12
Germany               3.493026e+12
France                2.681725e+12
Brazil                2.189793e+12
Italy                 2.120175e+12
India                 1.769298e+12
Canada                1.660648e+12
Russian Federation    1.565459e+12
Spain                 1.418079e+12
Australia             1.164041e+12
South Korea           1.106715e+12
Iran                  4.441559e+11
Name: Average GDP (2006-2015), dtype: float64

9. Створіть функцію, щоб визначити, наскільки змінився ВВП за останні 10 років для країни з 5-м середнім ВВП.

In [51]:
def gdp_change_for_5th_country(df):
    gdp_columns = [str(year) for year in range(2006, 2016)]
    avg_gdp_series = df[gdp_columns].mean(axis=1)
    avg_gdp_series.index = df.index
    avg_gdp_series = avg_gdp_series.sort_values(ascending=False)
    
    fifth_country = avg_gdp_series.index[4]
    
    gdp_change = df.loc[fifth_country, '2015'] - df.loc[fifth_country, '2006']
    
    return (fifth_country, gdp_change)

gdp_change_result = gdp_change_for_5th_country(final_data)

print(gdp_change_result)

('Brazil', np.float64(474340000000.0))


10. Створіть функцію, щоб визначити, в якій країні встановлено максимум поновлюваних джерел енергії (% Renewable) та який саме відсоток.

In [52]:
def max_renewable_country(df):
    max_renewable = df['% Renewable'].max() 
    country = df[df['% Renewable'] == max_renewable].index[0]
    return (country, max_renewable)

max_renewable_result = max_renewable_country(final_data)

print(max_renewable_result)

('Brazil', 69.64803)


11. Створіть стовпець, в якому оцінюється чисельність населення. Для пошуку оцінки використовуйте значення енергопостачання (Energy Supply) та енергопостачання на душу населення (Energy Supply per capita). Яка країна є шостою за цією оцінкою?

In [53]:
final_data['Estimated Population'] = final_data['Energy Supply'] / final_data['Energy Supply per Capita']

population_sorted = final_data.sort_values(by='Estimated Population', ascending=False)

sixth_country = population_sorted.index[5]
sixth_population = population_sorted.loc[sixth_country, 'Estimated Population']

def get_sixth_population_country(df):
    return (sixth_country, sixth_population)

sixth_population_result = get_sixth_population_country(final_data)

print(sixth_population_result)

('Germany', 80369696.96969697)


12. Створіть стовпець, в якому оцінюється кількість цитованих документів на одну особу. Який взаємозв'язок між кількістю цитованих документів на душу населення та енергозабезпеченням на душу населення?

In [54]:
final_data['Citations per Capita'] = final_data['Citations'] / final_data['Estimated Population']

correlation = final_data['Citations per Capita'].corr(final_data['Energy Supply per Capita'])

def calculate_correlation(df):
    return correlation

correlation_result = calculate_correlation(final_data)

print(correlation_result)

0.7485600772321211


13. Створіть новий стовпчик в який занесіть 1, якщо значення % відновлюваної енергії (% Renewable) країни дорівнює або вище медіани для всіх країн, і 0, якщо значення (% Renewable) нижче медіани.

In [57]:
renewable_median = final_data['% Renewable'].median()

final_data['High Renewable'] = (final_data['% Renewable'] >= renewable_median).astype(int)

sorted_data = final_data.sort_values(by='Rank')

def renewable_series(df):
    return df['High Renewable']

renewable_series_result = renewable_series(sorted_data)

display(renewable_series_result)

Country
China                 1
Japan                 0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                0
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: High Renewable, dtype: int64

In [56]:
ContinentDict = {
    'China':'Asia',
    'United States':'North America',
    'Japan':'Asia',
    'United Kingdom':'Europe',
    'Russian Federation':'Europe',
    'Canada':'North America',
    'Germany':'Europe',
    'India':'Asia',
    'France':'Europe',
    'South Korea':'Asia',
    'Italy':'Europe',
    'Spain':'Europe',
    'Iran':'Asia',
    'Australia':'Australia',
    'Brazil':'South America'
}

final_data['Continent'] = final_data.index.map(ContinentDict)

continent_stats = final_data.groupby('Continent')['Estimated Population'].agg(['size', 'sum', 'mean', 'std'])

def get_continent_statistics(df):
    return continent_stats

continent_statistics_result = get_continent_statistics(final_data)

display(continent_statistics_result)

,size,sum,mean,std
Continent,,,,
Asia,5,2898666386.6106,579733277.32212,6.790979e+08
Australia,1,23316017.316017,23316017.316017,NaN
Europe,5,394058699.474436,78811739.894887,3.813228e+07
North America,1,35239864.864865,35239864.864865,NaN
South America,1,205915254.237288,205915254.237288,NaN
